In [1]:
#Packages
import pandas as pd
import glob
import numpy as np
import warnings
from openpyxl import load_workbook
import os

#Creating Global Variables

#Input folder is where the intake data is stored.
input_folder = 'Input'

#Creating a dictinoary that will store all the dfs that will be sent to an excel file.
excel_dfs = {}

#Creating a list of excel sheets and a blank dataframe to create an excel sheets info 
    #which will hold a desciption of each excel sheet, since we have so many.
excel_sheets_info = pd.DataFrame(columns=['Excel Sheet', 'Description'])
excel_sheets_list = []

#We are appending it because we want it to appear first in the excel file:
excel_dfs["Excel Sheets Descriptions"] = excel_sheets_info

In [2]:
# List all Excel files in the folder
staff_evts_files = glob.glob(input_folder + '/' + 'Staffing_Events_Reports/' + '*.xlsx')

# Initialize an empty lists to store dataframes:
non_agcy_tsfr_term_dfs = []
employee_hr_actions_dfs = []

# Loop through each Excel file
#for file in staff_evts_files:
for file in staff_evts_files:
    #No Default style warning:
    with warnings.catch_warnings(record=True):
            warnings.simplefilter("always")
            # Read Excel file into a dataframe
            df = pd.read_excel(file, header=1)
        
            #Dropping blank values due to errors:
            df.dropna(subset=["Action/Reason"], inplace=True)

            # Filter rows where Employee Type is "Regular"
            df = df[df["Employee_Class"] == "Regular"]

            # Format "Effective Date" column as date
            df['Effective Date'] = pd.to_datetime(df['Effective Date'])
            # Create new column "CY" for Calendar Year
            df['CY'] = df['Effective Date'].dt.year
            # Create new column "QTR (CY)" for Quarter
            df['QTR (CY)'] = df['Effective Date'].dt.quarter

            #Extracting Department name out of Organization column:
            df['Department'] = df['Supervisory Organization'].apply(lambda x: x[x.find("340")+4:x.find("(")-1])

            #Contains the HR actions for each employee, this dataframe will be used to find the the department
                #based on the last HR action before an employee transfered out of our agency:
            employee_hr_actions_dfs.append(df[["Employee ID", "Worker", "Effective Date", "Department", 'Action/Reason']])
  
            # Filter rows containing "Termination" in the "Action/Reason" column:
            df = df[df['Action/Reason'].str.contains('Terminate')]

            #This is the first quarter where we can start caculating turnover from workday:
            df = df[df['Effective Date'] >= '2022-07-01']

            # Select only the required columns
            df = df[["CY", "QTR (CY)","Effective Date", "Employee ID", "Worker", "Department","Action/Reason"]]

            # Extracting Turnover Type
            df['Turnover Type'] = (df['Action/Reason'].str.extract(r'>\s*(.*?)\s*>'))
            # Extracting Turnover Reason
            df['Turnover Reason'] = df['Action/Reason'].str.split(' > ').str[-1]
            #Dropping Action/Reason because the column is redudant.
            df.drop(columns=['Action/Reason'], inplace=True)
        
            # Append filtered dataframe to the list
            non_agcy_tsfr_term_dfs.append(df)

# Concatenate all dataframes into one
non_agcy_tsfr_term_df = pd.concat(non_agcy_tsfr_term_dfs, ignore_index=True)
employee_hr_actions_df = pd.concat(employee_hr_actions_dfs, ignore_index=True)
#employee_hr_actions_df
#non_agcy_tsfr_term_df
#employee_hr_actions_df

Agecny Transfer Info:

In [3]:
# List all Excel files in the folder
agncy_tsfr_files = glob.glob(input_folder + '/' + 'Agency_Transfer_Reports/' + '*.xlsx')

# Initialize an empty list to store dataframes
agncy_tsfr_dfs = []

# Loop through each Excel file
for file in agncy_tsfr_files:
    # Read Excel file into a dataframe
    df = pd.read_excel(file)

    # Filter rows where Employee Type is "Regular", we only caculate Turnover for Regular Employees.
    df = df[df["Employee Type"] == "Regular"]

    # Format "Effective Date" column as date
    df['Effective Date'] = pd.to_datetime(df['Effective Date'])
    # Create new column "CY" for Calendar Year
    df['CY'] = df['Effective Date'].dt.year
    # Create new column "QTR (CY)" for Quarter
    df['QTR (CY)'] = df['Effective Date'].dt.quarter
    
    # Select only the required columns
    df = df[["CY", "QTR (CY)", "Effective Date", "Employee ID", "New Company"]]

    # Strip numeric characters and leading/trailing whitespace
    df['New Company'] = df['New Company'].str.replace(r'\d+', '', regex=True).str.strip()
    # Append filtered dataframe to the list
    agncy_tsfr_dfs.append(df)

# Concatenate all dataframes into one
agncy_tsfr_df = pd.concat(agncy_tsfr_dfs, ignore_index=True)
#agncy_tsfr_df

Extracting Department infomation for Employees who transfered to other agencies: 

In [4]:
#Create a list of Employee IDs who have transfered to other agencies
#tsfred stands for transfered
employee_ids_who_tsfred = agncy_tsfr_df['Employee ID'].unique().tolist()

# Filter employee_hr_actions_df where "Employee ID" is in employee_ids_who_tsfr
employee_hr_actions_who_tsfred_df = employee_hr_actions_df[employee_hr_actions_df['Employee ID'].isin(employee_ids_who_tsfred)]

match_records_df = pd.merge(agncy_tsfr_df, employee_hr_actions_who_tsfred_df, how='left', on='Employee ID', suffixes=('', ' of HR Action'))

#Because employee can transer in and back out this piece of code makes sure to pull the previous HR actions is 
    #before the transfer data. This prevents an HR action getting pulled back after their 1st transfer date. 
match_records_df = match_records_df[match_records_df['Effective Date'] > match_records_df['Effective Date of HR Action']]

# Calculating the difference in days between agnecy transfer date and last HR action:
match_records_df['Days Between Transfer and Previous HR Action'] = (match_records_df['Effective Date'] - match_records_df['Effective Date of HR Action']).dt.days

# Grouping by 'Employee ID' and 'Effective Date' and finding the index of the rows with the smallest 'Days Difference'
    #because we only want to pull back the most recent HR action before the transfer date.
idx = match_records_df.groupby(['Employee ID', 'Effective Date'])['Days Between Transfer and Previous HR Action'].idxmin()

#Applying the filter to get most recent HR action before the transfer date:
match_records_df = match_records_df.loc[idx]

# Reset the index if needed
match_records_df = match_records_df.reset_index(drop=True)

#We just need Effective Date and Employee ID to join and the only info we are wanting to know is the Department:
agncy_tsfr_with_dept = match_records_df[["Effective Date", "Employee ID", "Worker", "Department"]]

#Adding Final Columns so we concat the agncy_tsfr_df with non_agncy_tsfr_df
agncy_tsfr_df = pd.merge(agncy_tsfr_df, agncy_tsfr_with_dept, how='left', on=["Effective Date", "Employee ID"])
agncy_tsfr_df['Turnover Type'] = "Voluntary"
agncy_tsfr_df['Turnover Reason'] = "Agency Transfer"
#agncy_tsfr_df

Extracting Tenure infomation for all OSDH employees: 

In [5]:
#Creating a dataframe with the first HR action for all OSDH employees:
first_hr_actions_df = employee_hr_actions_df.loc[
    employee_hr_actions_df.groupby('Employee ID')['Effective Date'].idxmin()].reset_index(drop=True)
first_hr_actions_df = first_hr_actions_df[['Employee ID', 'Effective Date']]
# Rename the "Effective Date" column to "First Date in HR System"
first_hr_actions_df = first_hr_actions_df.rename(columns={'Effective Date': 'First Date in HR System'})                                             
#first_hr_actions_df

Finishing the Terminations Report/Dataframe: 

In [6]:
#Concat both terminations and agency transfers together:
OSDH_term_df = pd.concat([agncy_tsfr_df, non_agcy_tsfr_term_df], ignore_index=True)
#FY and finish CY setup:
OSDH_term_df['QTR (FY)'] = (OSDH_term_df['QTR (CY)'] + 2 - 1) % 4 + 1
OSDH_term_df['FY'] = OSDH_term_df.apply(lambda x: x['CY'] + 1 if x['Effective Date'].month in range(7, 13) else x['CY'], axis=1)
OSDH_term_df['FY QTR'] = OSDH_term_df['FY'].astype(str) + " Q" + OSDH_term_df['QTR (FY)'].astype(str)
OSDH_term_df['CY QTR'] = OSDH_term_df['CY'].astype(str) + " Q" + OSDH_term_df['QTR (CY)'].astype(str)

#Perform Merge and Caulations for agency Tenure:
OSDH_term_df = pd.merge(OSDH_term_df, first_hr_actions_df, how='left', on=["Employee ID"])
# Calculate the difference in years and create a new column "OSDH Tenure"
OSDH_term_df['OSDH Tenure'] = (OSDH_term_df['Effective Date'] - OSDH_term_df['First Date in HR System']).dt.days / 365.25
#Calculate OSDH Tenure in Months
OSDH_term_df['OSDH Tenure (Months)'] = OSDH_term_df['OSDH Tenure'] * 12

# Define the service year buckets
service_year_bins = [0, 1, 3, 5, 10, 20, float('inf')]
service_year_labels = ['<1 years', '1-3 years', '3-5 years', '5-10 years', '10-20 years', '20+ years']
# Create the "OSDH Tenure Buckets" column using pd.cut()
OSDH_term_df['OSDH Tenure Buckets'] = pd.cut(OSDH_term_df['OSDH Tenure'], bins=service_year_bins, labels=service_year_labels, right=False)

#We can come back and edit this line later but right now we only have completed data for FY 2024
OSDH_term_df = OSDH_term_df[OSDH_term_df['FY'] >= 2024]
#Apply remapping for cleaner visuials in Tabealu and due to lack of confidence in HR in assigning these values:
# List of reasons to remap to "Resignation"
reasons_to_remap = ['Moved', 'Accepted Other Employment', 'Resignation for Cause', 'Family Reasons']
# Remap values in the Turnover Reason column
OSDH_term_df['Turnover Reason'] = OSDH_term_df['Turnover Reason'].replace(reasons_to_remap, 'Resignation')
#Drop PI Data:
OSDH_term_df = OSDH_term_df.drop(columns=['Worker', 'Employee ID', 'Effective Date'])
#Reordering Dataframe for nicer output:
OSDH_term_df = OSDH_term_df[["FY QTR", "FY", "QTR (FY)", 
                             "CY QTR", "CY", "QTR (CY)", 
                             "Department", "Turnover Type", "Turnover Reason", "New Company",
                             "OSDH Tenure", "OSDH Tenure (Months)", "OSDH Tenure Buckets"]]
#OSDH_term_df
#PI_OSDH_term_df.to_clipboard(index=False)

Create the Turnover Headcount by Department Dataframe: 

Turnover Caculation Steps: 

In [7]:
def caculate_turnover(TO_folder, TO_breakdown_groups, excel_sheet, desc):
    #We use the Termination list to cut down on repetive calculation and chart displaying later on:
    Terminations = ['Involuntary', 'Voluntary', 'Total']

    # List all Excel files in the folder
    TO_summary_files = glob.glob(input_folder + '/' + TO_folder + '/'+ '*.xlsx')
    # Initialize an empty list to store dataframes
    TO_summary_dfs = []

    for file in TO_summary_files:
        with warnings.catch_warnings(record=True):
                warnings.simplefilter("always")
                # Read Excel file into a dataframe
                df = pd.read_excel(file, header=7)
                #Removing Sup Orgs with no active start or end heacounts that also have not terminations"
                df = df.loc[~((df['Start Headcount'] == 0) & (df['End Headcount'] == 0) & (df['Total Terminations'] == 0))]
    
                #Extracting Department name out of Organization column:
                df['Department'] = df['Organization'].apply(lambda x: x[x.find("340")+4:x.find("(")-1])
    
                # Loading in the day to get the FY that the report reflects:
                workbook = load_workbook(filename=file)
                sheet = workbook.active
                report_date = sheet['B7'].value

                if TO_folder == 'Turnover_Summary_FY_Reports':
                    df['FY'] = report_date.year
                else:
                    #We are pulling Turnover Summary Files on a Quarterly Basis and have to create code
                        #in order to create the correct Fiscal Year and Fiscal Quarter
                    df['End Date'] = pd.to_datetime(df['End Date'])
                    df['QTR (FY)'] = ((df['End Date'].dt.quarter) + 2 - 1) % 4 + 1
                    df['FY'] = df.apply(lambda x: report_date.year + 1 if x['End Date'].month in range(7, 13) else report_date.year, axis=1)
                    df['FY QTR'] = df['FY'].astype(str) + " Q" + df['QTR (FY)'].astype(str)

                #Performing Groupby to get start and end headcount for each department:
                df = df.groupby(TO_breakdown_groups).agg({'Start Headcount': 'sum', 'End Headcount': 'sum'}).reset_index()
                df['Average Headcount'] = ((df['Start Headcount'] + df['End Headcount']) / 2)
            
                #Filtering Down:
                #df = df[["FY", "Department", "Start Headcount", "End Headcount"]]
                # Append filtered dataframe to the list
                TO_summary_dfs.append(df)

    # Concatenate all dataframes into one
    TO_summary_df = pd.concat(TO_summary_dfs, ignore_index=True)

    #We can come back and edit this line later but right now we only have completed data for FY 2024
    TO_summary_df = TO_summary_df[TO_summary_df['FY'] >= 2024]

    #Getting Termination Numbers by TO_breakdown_groups:
    term_counts_df = (OSDH_term_df.groupby(TO_breakdown_groups)['Turnover Type'].value_counts().unstack(fill_value=0)).reset_index()
    #Rename columns to remove multi-level indexing and correct columns names
    term_counts_df.columns.name = None
    term_counts_df = term_counts_df.rename(columns={'Involuntary': 'Involuntary Terminations', 'Voluntary': 'Voluntary Terminations'})
    #Caculating Total Terminations
    term_counts_df["Total Terminations"] = term_counts_df['Involuntary Terminations'] + term_counts_df['Voluntary Terminations']
    
    #Merging Dataframesto have all the info needed to caculate Turnover:
    TO_df = pd.merge(TO_summary_df, term_counts_df, how='left', on=TO_breakdown_groups)
    # Some Department had no Terminations so we need to replace null values with zeros
    TO_df = TO_df.fillna(0)

    for t in Terminations:
            TO_df[t + ' Turnover Rate %'] = (TO_df[t + " Terminations"] / TO_df['Average Headcount'])
        
    #Appending Data to the excel_dfs dictionary:
    excel_dfs[excel_sheet] = TO_df
    
    #Assigning Excel Sheet Info Dataframe:
    new_excel_sheet_info = {'Excel Sheet': excel_sheet, 'Description': desc}

    # Append the new row data to the list
    excel_sheets_list.append(new_excel_sheet_info)
    
#pd.set_option('display.max_rows', None)

reg_emp_to_FY = 'Regular Employee Turnover by Fiscal Year'

caculate_turnover('Turnover_Summary_FY_Reports', ['FY'], 'TO_FY', reg_emp_to_FY)
caculate_turnover('Turnover_Summary_FY_QTR_Reports', ['FY QTR', 'FY', 'QTR (FY)'], 'TO_FY_QTR', reg_emp_to_FY + ' and by Quarter')
caculate_turnover('Turnover_Summary_FY_Reports', ['FY', 'Department'], 'TO_FY_DEPT', reg_emp_to_FY + ' and Department')

In [8]:
#Appending Termination Dataframe it will appear last:
excel_dfs["TERM_FY_QTR_DEPT"] = OSDH_term_df
excel_sheets_list.append({'Excel Sheet': "TERM_FY_QTR_DEPT", 
                          'Description': 'Individual Terminations by Fiscal Year, Quarter, and Department'})

#Concating all Excel Sheet Descriptions together:
excel_dfs["Excel Sheets Descriptions"] = pd.concat([excel_dfs["Excel Sheets Descriptions"],pd.DataFrame(excel_sheets_list)], ignore_index=True)

In [9]:
#Writing and Saving Data to excel file:
output_dir = 'Output - Update Regular Employee Turnover'
output_file_name = 'OSDH Regular Employee Turnover with Agency Transfers.xlsx'
out_file_path = os.path.join(output_dir, output_file_name)

writer = pd.ExcelWriter(out_file_path, engine='xlsxwriter')

#Setting Percent Format Up:
percent_fmt = {'num_format': '0.00%'}
percent_format = writer.book.add_format(percent_fmt)

# loop through `dict` of dataframes
for sheet, df in excel_dfs.items():  
    df.to_excel(writer, sheet_name=sheet, index=False) 
    
    #Saves Dataframe with filters turned on:
    worksheet = writer.sheets[sheet]
    worksheet.autofilter(0,0,df.shape[0],df.shape[1]-1)   
    
    #Adjusts sizes of columns for easier reading:
    for column in df:
        column_length = max(df[column].astype(str).map(len).max(), len(column) + 10) #9 for a bit of extra space
        col_idx = df.columns.get_loc(column)
        if '%' in column:
            writer.sheets[sheet].set_column(col_idx, col_idx, column_length, percent_format)
        else:
            writer.sheets[sheet].set_column(col_idx, col_idx, column_length)

writer.close()